# Classification Methods: Predicting Hotel Cancellations

### Python version

In [1]:
from platform import python_version
print(python_version())

3.8.2


### Import Libraries

In [2]:
import csv
import imblearn
import matplotlib.pyplot as plt
import mglearn
import numpy as np
from numpy.random import seed
seed(1)

import os
import pandas as pd
import random
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
from collections import Counter

### Import Data From AWS S3 to Sagemaker

In [3]:
# import boto3
# import botocore
# from sagemaker import get_execution_role

# role = get_execution_role()

# bucket = 'enterbucketname'
# data_key_train = 'H1full.csv'
# data_location_train = 's3://{}/{}'.format(bucket, data_key_train)

# train_df = pd.read_csv(data_location_train)

### Import Data Through CSV

In [4]:
train_df = pd.read_csv('new_train_data_akeshala_nipun_samal_new.csv')
validation_df = pd.read_csv('new_validation_data_akeshala_nipun_samal_new.csv')

In [5]:
a=train_df.head()
c = validation_df.head()


In [6]:
b=train_df
d = validation_df
train_df.describe

<bound method NDFrame.describe of        Unnamed: 0  Unnamed: 0.1  Reservation-id  Gender  Age  Ethnicity  \
0               0             0        39428300       1   40          1   
1               1             1        77491756       1   49          1   
2               2             2        73747291       1   42          2   
3               3             3        67301739       2   25          3   
4               4             4        77222321       1   62          1   
...           ...           ...             ...     ...  ...        ...   
27494       27494         27494        15645505       1   49          3   
27495       27495         27495        56414525       1   49          4   
27496       27496         27496        52023375       1   49          1   
27497       27497         27497        98041387       2   65          3   
27498       27498         27498        55243988       1   48          2   

       Educational_Level  Income  Country_region  Hotel_Type  ...

In [7]:

#b.sort_values(['Income','Previous_Cancellations'], ascending=True)

In [8]:
IsCanceled = train_df['Reservation_Status']
y_train = IsCanceled
y_validation = validation_df['Reservation_Status']
y_val = y_validation.to_numpy()
y_validation.shape
# y_train.describe
y_val.reshape(2749,1)
validation_df.shape

(2749, 26)

### Numerical Variables

In [9]:
Gender = train_df['Gender']
Age = train_df['Age']
#Ethnicity = train_df['Ethnicity']
Educational_Level = train_df['Educational_Level']
Income = train_df['Income']
Country_region = train_df['Country_region']
Hotel_Type = train_df['Hotel_Type']
No_of_Expected_Days_to_stay = train_df['No_of_Expected_Days_to_stay']
Checking_Month = train_df['Checking_Month']
#no_of_days_from_booking_to_ckeckin = train_df['no_of_days_from_booking_to_ckeckin']
Meal_Type = train_df['Meal_Type']
Visted_Previously = train_df['Visted_Previously']
Deposit_type = train_df['Deposit_type']
Booking_channel = train_df['Booking_channel']
Required_Car_Parking = train_df['Required_Car_Parking']
Use_Promotion = train_df['Use_Promotion']
Discount_Rate = train_df['Discount_Rate']
Room_Rate = train_df['Room_Rate']
adults = train_df['Adults']
children = train_df['Children']
babies = train_df['Babies']
previouscancellations = train_df['Previous_Cancellations']

In [10]:
x = np.column_stack((Gender,Age,Educational_Level,Income,Country_region,Hotel_Type,No_of_Expected_Days_to_stay,Checking_Month,Meal_Type,Visted_Previously,Deposit_type,Booking_channel,Required_Car_Parking,Use_Promotion,Discount_Rate,Room_Rate,adults,children,babies,previouscancellations))
x = sm.add_constant(x, prepend=True)
x.shape

(27499, 21)

### Validation Numerical Variables

In [11]:
Gender_v = validation_df['Gender']
Gender_v.shape
Age_v = validation_df['Age']
#Ethnicity_v = validation_df['Ethnicity']
Educational_Level_v = validation_df['Educational_Level']
Income_v = validation_df['Income']
Country_region_v = validation_df['Country_region']
Hotel_Type_v = validation_df['Hotel_Type']
No_of_Expected_Days_to_stay_v = validation_df['No_of_Expected_Days_to_stay']
Checking_Month_v = validation_df['Checking_Month']
#no_of_days_from_booking_to_ckeckin_v = validation_df['no_of_days_from_booking_to_ckeckin']
Meal_Type_v = validation_df['Meal_Type']
Visted_Previously_v = validation_df['Visted_Previously']
Deposit_type_v = validation_df['Deposit_type']
Booking_channel_v = validation_df['Booking_channel']
Required_Car_Parking_v = validation_df['Required_Car_Parking']
Use_Promotion_v = validation_df['Use_Promotion']
Discount_Rate_v = validation_df['Discount_Rate']
Room_Rate_v = validation_df['Room_Rate']
adults_v = validation_df['Adults']
children_v = validation_df['Children']
babies_v = validation_df['Babies']
previouscancellations_v = validation_df['Previous_Cancellations']

In [12]:
x_v = np.column_stack((Gender_v,Age_v,Educational_Level_v,Income_v,Country_region_v,Hotel_Type_v,No_of_Expected_Days_to_stay_v,Checking_Month_v,Meal_Type_v,Visted_Previously_v,Deposit_type_v,Booking_channel_v,Required_Car_Parking_v,Use_Promotion_v,Discount_Rate_v,Room_Rate_v,adults_v,children_v,babies_v,previouscancellations_v))
x_v = sm.add_constant(x_v, prepend=True)
x_v.shape

(2749, 21)

## Feature Selection

### Wrapper-Based: Forward Search

In [13]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import roc_auc_score
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.preprocessing import StandardScaler

#  forward_feature_selector = SequentialFeatureSelector(RandomForestClassifier(n_jobs=-1),
#            k_features=6,
#            forward=True,
#             verbose=2,
#            scoring='roc_auc',
#             cv=4)

In [14]:
scaler = StandardScaler()
X_train = scaler.fit_transform(x)
X_test = scaler.transform(x_v)
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 42)

In [15]:
classifier.fit(x, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [16]:
y_predict = classifier.predict(X_test)

In [17]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_validation,y_predict))
print(classification_report(y_validation,y_predict))

[[   0    0  741]
 [   1    0  397]
 [   0    0 1610]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       741
           1       0.00      0.00      0.00       398
           2       0.59      1.00      0.74      1610

    accuracy                           0.59      2749
   macro avg       0.20      0.33      0.25      2749
weighted avg       0.34      0.59      0.43      2749



C:\Users\dewth\anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Wrapper-Based: Backward Search

In [18]:
# from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
# from sklearn.metrics import roc_auc_score 

# backward_feature_selector = SequentialFeatureSelector(RandomForestClassifier(n_jobs=-1),
#            k_features=6,
#            forward=False,
#            verbose=2,
#            scoring='roc_auc',
#            cv=4)

In [19]:
# bselector = backward_feature_selector.fit(x, y)

In [20]:
# bselector.k_feature_names_

### Extra Trees Classifier

In [21]:
# from sklearn.ensemble import ExtraTreesClassifier
# model = ExtraTreesClassifier()
# model.fit(x, y)
# print(model.feature_importances_)

In [22]:
# ext=pd.DataFrame(model.feature_importances_,columns=["extratrees"])
# ext
# ext.sort_values(['extratrees'], ascending=True)

### Selected Features

In [23]:
# x1 = np.column_stack((leadtime,countrycat,marketsegmentcat,deposittypecat,customertypecat,rcps,arrivaldateweekno))
# x1 = sm.add_constant(x1, prepend=True)

In [24]:
# x1_train, x1_val, y1_train, y1_val = train_test_split(x1, y, random_state=0)

### SVM

In [25]:
# https://stackoverflow.com/questions/52896387/svc-with-class-weight-in-scikit-learn
from sklearn import svm

clf = svm.LinearSVC(penalty='l2', loss='squared_hinge', dual=True, tol=0.0001, C=1.0, multi_class='ovr', fit_intercept=True, intercept_scaling=1, class_weight=None, verbose=0, random_state=None, max_iter=5000)
clf.fit(X_train, y_train)  
prclf = clf.predict(X_test)


In [26]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_val,prclf))
print(classification_report(y_val,prclf))

[[   0    0  741]
 [   0    0  398]
 [   0    0 1610]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       741
           1       0.00      0.00      0.00       398
           2       0.59      1.00      0.74      1610

    accuracy                           0.59      2749
   macro avg       0.20      0.33      0.25      2749
weighted avg       0.34      0.59      0.43      2749



C:\Users\dewth\anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [27]:
print(type(prclf))
print(type(y_val))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


### XGBOOST

In [ ]:
import xgboost as xgb
xgb_model = xgb.XGBClassifier(learning_rate=0.00001,
                            max_depth =20, 
                            n_estimators = 50,use_label_encoder=False,eval_metric = 'mlogloss' )
xgb_model.fit(x, y_train)

print("Accuracy on training set: {:.3f}".format(xgb_model.score(x, y_train)))
print("Accuracy on validation set: {:.3f}".format(xgb_model.score(x_v, y_val)))

In [ ]:
xgb_predict=xgb_model.predict(X_test)
xgb_predict

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_val,xgb_predict))
print(classification_report(y_val,xgb_predict))

### Oversampling for Naive Bayes and KNN Models

In [ ]:
counter = Counter(y_train)
print(counter)

In [ ]:
oversample = SMOTE()
X_train, y_train = oversample.fit_resample(X_train, y_train)

In [ ]:
counter = Counter(y_train)
print(counter)

### Naive Bayes

In [ ]:
gnb = GaussianNB()
gnb

In [ ]:
y_pred = gnb.fit(X_train, y_train).predict(X_test)
y_pred

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_val,y_pred))
print(classification_report(y_val,y_pred))

### KNN and SMOTE Oversampling

In [ ]:
x1_train = MinMaxScaler().fit_transform(X_train)

In [ ]:
x1_train

In [ ]:
knn = KNeighborsClassifier(n_neighbors=10)
knn=knn.fit(x1_train, y_train)
pred = knn.predict(X_test)
pred
print("Training set score: {:.2f}".format(knn.score(x1_train, y_train)))
print("Validation set score: {:.2f}".format(knn.score(X_test, y_val)))

# KNN Plot
mglearn.plots.plot_knn_classification(n_neighbors=10)
plt.show()

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_val,pred))
print(classification_report(y_val,pred))

## Test Data using SVM Model

### Import Data from AWS S3 to SageMaker

In [ ]:
# data_key_test = 'H2full.csv'
# data_location_test = 's3://{}/{}'.format(bucket, data_key_test)

# h2data = pd.read_csv(data_location_test)

### Import CSV

In [ ]:
h2data = pd.read_csv('H2.csv')
a=h2data.head()
a

In [ ]:
type(h2data)

In [ ]:
t_leadtime = h2data['LeadTime'] #1
t_arrivaldateyear = h2data['ArrivalDateYear']
t_arrivaldateweekno = h2data['ArrivalDateWeekNumber']
t_arrivaldatedayofmonth = h2data['ArrivalDateDayOfMonth']
t_staysweekendnights = h2data['StaysInWeekendNights'] #2
t_staysweeknights = h2data['StaysInWeekNights'] #3
t_adults = h2data['Adults'] #4
t_children = h2data['Children'] #5
t_babies = h2data['Babies'] #6
t_previouscancellations = h2data['PreviousCancellations'] #12
t_previousbookingsnotcanceled = h2data['PreviousBookingsNotCanceled'] #13
t_bookingchanges = h2data['BookingChanges'] #16
t_dayswaitinglist = h2data['DaysInWaitingList'] #20
t_adr = h2data['ADR'] #22
t_rcps = h2data['RequiredCarParkingSpaces'] #23
t_totalsqr = h2data['TotalOfSpecialRequests'] #24

In [ ]:
t_arrivaldatemonth = h2data.ArrivalDateMonth.astype("category").cat.codes
t_arrivaldatemonthcat = pd.Series(t_arrivaldatemonth)
t_mealcat=h2data.Meal.astype("category").cat.codes
t_mealcat=pd.Series(t_mealcat)
t_countrycat=h2data.Country.astype("category").cat.codes
t_countrycat=pd.Series(t_countrycat)
t_marketsegmentcat=h2data.MarketSegment.astype("category").cat.codes
t_marketsegmentcat=pd.Series(t_marketsegmentcat)
t_distributionchannelcat=h2data.DistributionChannel.astype("category").cat.codes
t_distributionchannelcat=pd.Series(t_distributionchannelcat)
t_reservedroomtypecat=h2data.ReservedRoomType.astype("category").cat.codes
t_reservedroomtypecat=pd.Series(t_reservedroomtypecat)
t_assignedroomtypecat=h2data.AssignedRoomType.astype("category").cat.codes
t_assignedroomtypecat=pd.Series(t_assignedroomtypecat)
t_deposittypecat=h2data.DepositType.astype("category").cat.codes
t_deposittypecat=pd.Series(t_deposittypecat)
t_customertypecat=h2data.CustomerType.astype("category").cat.codes
t_customertypecat=pd.Series(t_customertypecat)
t_reservationstatuscat=h2data.ReservationStatus.astype("category").cat.codes
t_reservationstatuscat=pd.Series(t_reservationstatuscat)
t_isrepeatedguestcat = h2data.IsRepeatedGuest.astype("category").cat.codes
t_isrepeatedguestcat=pd.Series(t_isrepeatedguestcat)
t_agentcat = h2data.Agent.astype("category").cat.codes
t_agentcat=pd.Series(t_agentcat)
t_companycat = h2data.Company.astype("category").cat.codes
t_companycat=pd.Series(t_companycat)

In [ ]:
a = np.column_stack((t_leadtime,t_countrycat,t_marketsegmentcat,t_deposittypecat,t_customertypecat,t_rcps,t_arrivaldateweekno))
a = sm.add_constant(a, prepend=True)
IsCanceled = h2data['IsCanceled']
b = IsCanceled
b=b.values

In [ ]:
prh2 = clf.predict(a)
prh2

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(b,prh2))
print(classification_report(b,prh2))